## 필요한 라이브러리 가져오기

In [596]:
import os
import bs4
from dotenv import load_dotenv
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langsmith import Client

## 환경 변수 가져와서 API key 설정하기

In [598]:
load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
langsmith_api_key = os.getenv("LANGSMITH_API_KEY")
langsmith_endpoint = os.getenv("LANGSMITH_ENDPOINT")

os.environ["USER_AGENT"] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36"
os.environ["LANGCHAIN_API_KEY"] = langsmith_api_key
os.environ["LANGCHAIN_ENDPOINT"] = langsmith_endpoint
os.environ["LANGSMITH_TRACING"] = "true"

## 모델 설정 : gpt-4o-mini

In [600]:
llm = ChatOpenAI(model="gpt-4o-mini", api_key=api_key)
client = Client(api_key=langsmith_api_key)

## Knowledge source : 스파르타 공모전 수상작 페이지 크롤링

### WebBaseLoader
 #### web_paths (or web_path)
   - load할 web 페이지 경로
   - 스파르타 공모전 수상작 페이지로 설정 : "https://spartacodingclub.kr/blog/all-in-challenge_winner"  

 #### bs_kwargs
   - beautifulsoup4가 받는 key arguments

### BeautifulSoup4
 #### 웹사이트 크롤링 라이브러리
  ##### class SoupStrainer
   - parse_only : document의 특정 부분만 parsing할지 정할 수 있음
   - 특정 tag나 class를 설정할 수 있음
   - 위 웹페이지에서 본문 영역인 "css-8lvslw" class를 parsing

In [602]:
loader = WebBaseLoader(
    web_paths=("https://spartacodingclub.kr/blog/all-in-challenge_winner",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("css-8lvslw") # "css-8lvslw" class 선
        )
    ),
)
docs = loader.load()

In [603]:
docs

[Document(metadata={'source': 'https://spartacodingclub.kr/blog/all-in-challenge_winner'}, page_content="스파르타 소식'AII-in 코딩 공모전’ 수상작을 소개합니다조회수  778·6분 분량2024. 9. 3.코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적인 서비스를 만들어 세상을 바꿀 수 있습니다. 스파르타코딩클럽에서는 이러한 가능성을 믿고, 누구나 코딩을 통해 자신의 아이디어를 실현하고 실제 문제를 해결하는 경험을 쌓을 수 있도록 다양한 프로그램을 마련하고 있습니다.<All-in> 코딩 공모전은 대학생들이 캠퍼스에서 겪은 불편함과 문제를 자신만의 아이디어로 해결해보는 대회였는데요. 이번 공모전에서 다양한 혁신적인 아이디어와 열정으로 가득한 수많은 프로젝트가 탄생했습니다. 그중 뛰어난 성과를 낸 수상작 6개를 소개합니다.🏆\xa0대상[Lexi Note] 언어공부 필기 웹 서비스서비스 제작자: 다나와(김다애, 박나경)💡W는 어문학을 전공하는 대학생입니다. 매일 새로운 단어와 문장 구조를 공부하고 있지만, 효율적으로 학습하는 것이 쉽지 않았습니다. 단어의 의미를 찾기 위해 사전을 뒤적이고, 긴 문장을 이해하려고 번역기를 사용하다 보면, 필기 노트는 어느새 뒷전으로 밀려났거든요. 사전, 번역기, 원서, 필기노트를 왔다 갔다 하다 보면 시간이 다 지나가 버리곤 했죠.W와 같이 어문 전공생은 문법, 어휘, 문장 구조 등 다양한 자료를 학습해야 합니다. 여러 자료를 번갈아 학습하다보니 ‘사전-번역기-원서-필기노트’ 왕복으로 학습 효율이 나지 않아 고민인 경우도 많으실 거예요. <Lexi Note>는 단어를 드래그하면 네이버 사전으로 바로 연동 돼 단어의 의미를 찾으며 동시에 필기 할 수 있어요. 이외에도 번역 버튼을 누르면 파파고 번역기가 연동돼 긴 문장도 쉽게 이해할 수 있어요. 언어 학습에 필요한 할일 목록과 스케줄 템플릿을 제공하여 효율적으로 공부할 수 

## Knowledge source에서 text 추출

### 대상작만 요약하는 문제가 있음

#### i) chunk_size = 1000자로 설정한 경우
  - load할 때는 전체 작품들이 포함되어 있음
  - prompt를 넣어줬을 때, 대상 1개만 요약함 

#### ii) chunk_size = 3000자로 설정한 경우
  - prompt를 넣어줬을 때, 우수상 2개, 입선 1개까지 요약함  

#### iii) chunk_size = 5000자로 설정한 경우
  - prompt를 넣어줬을 때, 대상 1개, 우수상 2개, 입선 3개 모두 요약함  
  - 글 후반부에 수상작들을 요약하는 부분까지 추가됨

#### chunk_overlap
  - 인접한 두 chunk가 겹치도록 설정하여 context를 유지

### Vector store
#### - Chroma로 vector store 생성
#### - vectorstore는 text와 embedding의 pair

In [605]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=5000, # chunk_size를 1000자 => 5000자로 늘림
    chunk_overlap=200 # 두 chunk가 200자 겹치도록 설정
)
splits = text_splitter.split_documents(docs)

vectorstore = Chroma.from_documents(
    documents=splits, 
    embedding=OpenAIEmbeddings(api_key=api_key)
)

## 질문과 유사한 embedding을 갖고 있는 text를 추출하는 코드

### vector store에서 retreiver 생성
#### top k 설정하기
  - 검색해서 return할 유사한 문서, chunk의 수를 결정하는 부분
  - default는 4로 되어 있음

##### i) k = 4 (default)
  - chunk size : 1000, 3000 => 대상 1개, 우수상 2개만 요약함
  - chunk size : 5000 => 대상 1개, 우수상 2개, 입선 1개까지 요약함
##### ii) k = 7
  - chunk size : 1000, 3000, 5000 => 대상 1개, 우수상 2개, 입선 1개까지 요약함
##### iii) k = 10
  - chunk size : 1000 => 대상 1개, 우수상 2개까지 요약함
  - chunk size : 3000 => 대상 1개, 우수상 2개, 입선 1개까지 요약함
  - chunk size : 5000 => 전부 다 요약할 때도 있고 아닐 때도 있음

##### k값을 늘리는 것의 의미
  - k값이 작으면 문서가 여러 chunk로 쪼개진 경우 앞 부분만 검색될 수 있는데, 이 부분을 방지할 수 있음
  - k값이 작으면 문서의 일부만 보고 편향된 응답을 할 수 있는데 그럴 가능성이 낮아짐
  - k값이 너무 크면 관련없는 정보까지 포함될 수 있음

In [607]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

## rlm/rag-prompt
### 기본 template로 인한 문제
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

  - 모르면 모른다고 대답하고 최대 3문장으로만 요약하도록 설정되어 있음
  - 그래서 chunk size를 늘리고 k 값을 늘려도 수상작 전체가 포함될 때도 있지만 대부분 포함되지 않음

In [609]:
from langchain import hub

user_msg = "ALL-in 코딩 공모전 수상작들을 요약해줘."
retrieved_docs = retriever.invoke(user_msg)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

prompt = hub.pull("rlm/rag-prompt")

In [610]:
user_prompt = prompt.invoke({"context": format_docs(retrieved_docs), "question": user_msg})
print(user_prompt)

messages=[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: ALL-in 코딩 공모전 수상작들을 요약해줘. \nContext: 스파르타 소식'AII-in 코딩 공모전’ 수상작을 소개합니다조회수  778·6분 분량2024. 9. 3.코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적인 서비스를 만들어 세상을 바꿀 수 있습니다. 스파르타코딩클럽에서는 이러한 가능성을 믿고, 누구나 코딩을 통해 자신의 아이디어를 실현하고 실제 문제를 해결하는 경험을 쌓을 수 있도록 다양한 프로그램을 마련하고 있습니다.<All-in> 코딩 공모전은 대학생들이 캠퍼스에서 겪은 불편함과 문제를 자신만의 아이디어로 해결해보는 대회였는데요. 이번 공모전에서 다양한 혁신적인 아이디어와 열정으로 가득한 수많은 프로젝트가 탄생했습니다. 그중 뛰어난 성과를 낸 수상작 6개를 소개합니다.🏆\xa0대상[Lexi Note] 언어공부 필기 웹 서비스서비스 제작자: 다나와(김다애, 박나경)💡W는 어문학을 전공하는 대학생입니다. 매일 새로운 단어와 문장 구조를 공부하고 있지만, 효율적으로 학습하는 것이 쉽지 않았습니다. 단어의 의미를 찾기 위해 사전을 뒤적이고, 긴 문장을 이해하려고 번역기를 사용하다 보면, 필기 노트는 어느새 뒷전으로 밀려났거든요. 사전, 번역기, 원서, 필기노트를 왔다 갔다 하다 보면 시간이 다 지나가 버리곤 했죠.W와 같이 어문 전공생은 문법, 어휘, 문장 구조 등 다양한 자료를 학습해야 합니다. 여러 자료를 번갈아 학습하다보니 

In [611]:
response = llm.invoke(user_prompt)
print(response.content)

ALL-in 코딩 공모전에서는 다양한 혁신적인 프로젝트가 선정되었으며, 대표적인 수상작으로는 'Lexi Note'라는 언어 공부 필기 웹 서비스가 있습니다. 이 서비스는 사용자가 단어를 드래그하여 즉시 사전과 연동되도록 하고, 효율적인 학습을 도와주는 기능을 제공합니다. 또한, '우리집 히어로즈'와 '에코 클래스룸' 등의 우수상 수상작들은 각각 벌레 퇴치와 수업 소통 문제를 해결하기 위한 서비스입니다.


### custom_prompt 설정
#### 1. 전체 수상작을 포함시키도록 수정
#### 2. 각 수상작에 대해 1문장으로 요약하도록 설정
#### 3. 최대 8문장으로 제한
#### 4. 이어지는 문단으로 기술할 것
#### 5. 한글로 요약하되 프로젝트명이 영어이면 영어로 표기

In [648]:
from langchain.prompts import PromptTemplate

custom_prompt = PromptTemplate.from_template("""
You are an assistant for summarizing competition results.

Using the context below, write a single paragraph summarizing each of the 6 award-winning projects** from the competition.
Write exactly one sentence per project in Kroean and project name should be in English if it is.
Do not use bullet points or line breaks.
Make sure all 6 projects are clearly mentioned. Total: 8 sentences maximum.

Question: {question}

Context:
{context}

Answer:
""")

In [650]:
user_prompt = custom_prompt.invoke({"context": format_docs(retrieved_docs), "question": user_msg})
print(user_prompt)

text="\nYou are an assistant for summarizing competition results.\n\nUsing the context below, write a single paragraph summarizing each of the 6 award-winning projects** from the competition.\nWrite exactly one sentence per project in Kroean and project name should be in English if it is.\nDo not use bullet points or line breaks.\nMake sure all 6 projects are clearly mentioned. Total: 8 sentences maximum.\n\nQuestion: ALL-in 코딩 공모전 수상작들을 요약해줘.\n\nContext:\n스파르타 소식'AII-in 코딩 공모전’ 수상작을 소개합니다조회수  778·6분 분량2024. 9. 3.코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적인 서비스를 만들어 세상을 바꿀 수 있습니다. 스파르타코딩클럽에서는 이러한 가능성을 믿고, 누구나 코딩을 통해 자신의 아이디어를 실현하고 실제 문제를 해결하는 경험을 쌓을 수 있도록 다양한 프로그램을 마련하고 있습니다.<All-in> 코딩 공모전은 대학생들이 캠퍼스에서 겪은 불편함과 문제를 자신만의 아이디어로 해결해보는 대회였는데요. 이번 공모전에서 다양한 혁신적인 아이디어와 열정으로 가득한 수많은 프로젝트가 탄생했습니다. 그중 뛰어난 성과를 낸 수상작 6개를 소개합니다.🏆\xa0대상[Lexi Note] 언어공부 필기 웹 서비스서비스 제작자: 다나와(김다애, 박나경)💡W는 어문학을 전공하는 대학생입니다. 매일 새로운 단어와 문장 구조를 공부하고 있지만, 효율적으로 학습하는 것이 쉽지 않았습니다. 단어의 의미를 찾기 위해 사전을 뒤적이고, 긴 문장을 이해하려고 번역기를 사용하다

In [652]:
response = llm.invoke(user_prompt)
print(response.content)

이번 ALL-in 코딩 공모전의 대상인 [Lexi Note]는 언어 전공 학생들이 효율적으로 어휘를 학습할 수 있도록 단어 검색, 번역, 필기 기능을 통합하여 학습 속도를 개선하는 웹 서비스입니다. 우수상인 [우리집 히어로즈]는 자취생들이 벌레 문제를 해결하기 위해 학생들 간의 매칭을 통해 안전하게 벌레를 퇴치할 수 있는 앱으로, 신원 보장을 통해 사용자들이 불안감을 느끼지 않도록 돕습니다. 또 다른 우수상인 [에코 클래스룸]은 학생들이 익명으로 질문을 제출하고 교수는 이를 바탕으로 수업 내용을 조정할 수 있어 실시간 소통을 가능하게 하는 혁신적인 플랫폼입니다. 입선작 [Crewing]은 신입생들이 원하는 연합 동아리를 쉽게 찾고 가입할 수 있도록 정보를 제공하는 아카이빙 플랫폼으로, 신뢰할 수 있는 동아리 정보를 모아주는 기능을 갖추고 있습니다. [Chore Buddy]는 대학생들이 집안일을 함께 할 사람을 매칭해주는 서비스로, 함께 생활하는 재미를 더해주는 기능을 제공합니다. 마지막으로 [StudyPal]은 친구와 함께 학습할 수 있는 세션을 예약하고, 서로의 공부 계획을 공유하여 동기를 부여하는 소셜 학습 플랫폼으로, 서로의 학업을 지원하는 기회를 제공합니다.
